In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

!pip install transformers
!pip install contractions

from transformers import AutoTokenizer,AutoModelWithLMHead
import contractions

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 13.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import numpy as np

In [ ]:
#Load Dataset
df = pd.read_csv('quotes.csv', header=None, names = ['Quote', 'Author', 'Categories'], engine = 'python', on_bad_lines = 'skip')
df = df[:30000]

df = df.iloc[:, 0:3]
#
df.describe()

searchfor = ['inspiration', 'motivation', 'life-lesson', 'love', 'hope', 'friendship', 'life', 'faith', 'universe', 'nature']
df = df[df['Categories'].str.contains('|'.join(searchfor),na=False)]
df['Length'] = df['Quote'].str.split().apply(len)
df.describe()


,Length
count,7177.000000
mean,38.935906
std,49.800967
min,1.000000
25%,14.000000
50%,23.000000
75%,44.000000
max,700.000000


In [ ]:
train_test_ratio = 0.85
train_valid_ratio = 70/85
df_full_train, df_test = train_test_split(df, train_size = train_test_ratio, random_state = 1)
# df_train, df_valid = train_test_split(df_full_train, train_size = train_valid_ratio, random_state = 1)

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("gpt2")

def prepare_dataset(df, dest_path):
    f = open(dest_path, 'w')
    data = ''
    quotes = df['Quote'].tolist()
    print(len(quotes))
    for quote in quotes:
        #Making all words lower case
        quote = str(quote).strip().lower()

        #Removing extra whitespaces
        quote = re.sub(r"\s", " ", quote)

        #Expand contractions
        quote = contractions.fix(quote)

        #Filtering out quotes longer than 35 words and less than 10
        if(len(quote.split())<10 or len(quote.split())>100):
          continue

        #Remove non-ASCII characters
        encoded_string = quote.encode("ascii", "ignore")
        quote = encoded_string.decode()

        #Remove string with these characters
        searchfor = ['~', '-', '--']
        match = False
        for s in searchfor:
          if(s in quote):
            match = True
            break
        if(match):
          continue

        #Insert whitespace between word and punctuation
        # print(quote)
        quote = re.sub(r"([\w/'+$\s-]+|[^\w/'+$\s-]+)\s*", r"\1 ", quote)
        quote = re.sub(r'<[^>]+>', r"",quote)
        # print(quote)
        data += tokenizer.special_tokens_map['bos_token']+quote+tokenizer.special_tokens_map['eos_token']+'\n'

    f.write(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
prepare_dataset(df_full_train, 'train.txt')
prepare_dataset(df_test, 'test.txt')
# prepare_dataset(df_valid, 'valid.txt')

6100
1077


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelWithLMHead.from_pretrained('distilgpt2')

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset('train.txt','test.txt',tokenizer)


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (182123 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./models", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, # number of training epochs
    per_gpu_train_batch_size=32, # batch size for training
    per_gpu_eval_batch_size=64,  # batch size for evaluation
    logging_steps = 500, # Number of update steps between two evaluations.
    save_steps=5000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # prediction_loss_only=True,
    # compute_metrics=True
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:

trainer.train()
trainer.save_model("./models")
tokenizer.save_pretrained("./models")

from google.colab import files
files.download('./models/pytorch_model.bin')



Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,3.552100
1000,2.872700
1500,2.501600
2000,2.271900


FileNotFoundError: Cannot find file: ./models/pytorch_model.bin

# Inference

In [ ]:
!pip install transformers
from transformers import *

tokenizer = AutoTokenizer.from_pretrained("./models")
model = AutoModelWithLMHead.from_pretrained("./models")

gpt2_finetune = pipeline('text-generation', model=model, tokenizer=tokenizer)


In [ ]:
def generate_quote(starting_text, min_length, max_length ):
  gen_text = gpt2_finetune (starting_text, min_length = min_length, max_length= max_length, top_k=50, top_p=0.95, temperature=0.5)
  return gen_text[0]['generated_text']





In [ ]:
temp_quotes = []
for i in range(0, len(all_quotes), 10):
  lowest_index = 0;
  for j in range(10):
    if len(all_quotes[i + j]) < len(all_quotes[i + lowest_index]):
      lowest_index = j

  temp_quotes.append(all_quotes[i + lowest_index])

for q in temp_quotes:
  print(q)

## Generating the images

In [ ]:
from google.colab import drive
from PIL import Image, ImageDraw, ImageFont
import random
import os

# Mount Google Drive
drive.mount('/content/drive')




In [ ]:


def add_line_breaks(text, word_count):
    words = text.split()  # Split the text into individual words
    output_text = ""
    line_length = 0
    for word in words:
        if line_length == word_count-1:
            output_text += word + "\n"
            line_length = 0
        else:
            output_text += word + " "
        line_length += 1

    return output_text


In [ ]:

def decrease_brightness(image, factor):

    if image.mode != 'RGB':
        image = image.convert('RGB')

    darkened_image = Image.blend(image, Image.new('RGB', image.size, 'black'), factor)

    return darkened_image

In [ ]:
# Update the image path to point to the folder in your Google Drive


In [ ]:

def generate_image(quote):
  image_path = '/content/drive/MyDrive/NLPproject/Images'
  font_path = '/content/drive/MyDrive/NLPproject/Fonts'

  # Get a list of all the files in the folder
  Font_files = os.listdir(font_path)
  Image_files = os.listdir(image_path)

  # Filter the list to only include image files
  filtered_font_files = [f for f in Font_files if f.endswith(('.ttf'))]
  filtered_image_files = [f for f in Image_files if f.endswith(('.png','.jpg','.jpeg'))]

  # Randomly pick an image file from the list
  random_font = random.choice(filtered_font_files)
  random_image = random.choice(filtered_image_files)

  # Open image
  image = Image.open(os.path.join(image_path, random_image))
  image = decrease_brightness(image,0.4)

  # Create drawing object and font
  draw = ImageDraw.Draw(image)
  font_path = os.path.join(font_path, random_font)

  # Define text and random position
  text = quote
  text = add_line_breaks(text, 6)

  print(text)
  # Calculate the maximum font size based on image dimensions

  # Choose a random font size within the maximum limit
  font_size = int(image.width/len(text)*2)

  font = ImageFont.truetype(font_path, font_size)

  # Calculate the width and height of the text
  text_width, text_height = draw.textsize(text, font=font)

  # Calculate the random position within the image to place the text
  x_pos = random.randint(0, image.width - text_width)
  y_pos = random.randint(0, image.height - text_height)

  # Draw text on image at random position
  draw.text((x_pos, y_pos), text, font=font, fill=(255, 255, 255))

  display(image)

In [ ]:
prompts = ['why',
'the thing about',
'Life is',
'In the end ',
'Happiness is ',
'No one',
'You are',
'Love is',
'The',
'Believe',
'Dream',
'Success',
'Inspire',
'Courage',
'Achieve',
'Believe in yourself',
'Motivation',
'Persistence',
'Passion',
'Strength',
'Positivity',
'Change',
'Hope',
'Empower',
]

prompt = random.choice(prompts)

for i in range(25):
  generate_image(generate_quote(prompt, 10, 50))